# LSTM Training

In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [2]:
import pandas as pd
import numpy as np
import itertools
import os
import random

In [3]:
# read finished Data; case training
def readData(src):
    dataseries = []
    for i in itertools.count():
        try:
            #print(i)
            dataseries.append(np.array(pd.read_json(src+ '/'+os.path.basename(src[:-4])+'_'+str(i).zfill(12)+'_keypoints.json').iloc[0,0]['pose_keypoints_2d']))
        except ValueError:
            break
        except IndexError:
            dataseries.append(np.zeros(25*3))
    return dataseries,i

In [4]:
labelNames=['rennen','laufen']
dataPaths=['data/output/rennen/','data/output/laufen/']
samplePaths=[]
data=[]
labels=[]

dirs0 = os.listdir(dataPaths[0])
for p in dirs0:
    samplePaths.append(dataPaths[0]+p+'0')
dirs1 = os.listdir(dataPaths[1])
for p in dirs1:
    samplePaths.append(dataPaths[1]+p+'1')


random.shuffle(samplePaths)
#print(samplePaths)
minLength = 75
for smp in samplePaths:
    l = smp[-1:]
    smp = smp[:-1]
    td, i = readData(smp)
    #if i<minLength:
    #    minLength = i
    data.append(td[:minLength])
    if l == '0':
        labels.append(np.array([1,0]))
    elif l == '1':
        labels.append(np.array([0,1]))
    else:
        #labels.append(np.array([0,0]))
        print("can't load: ",smp)
    print('.', end="")
        
print("data loaded.")
print("min Length: ", minLength)

.............................................data loaded.
min Length:  75


In [5]:
#minLength

In [6]:
#define constants

time_steps=minLength
#hidden LSTM units
num_units=128
# 25*3=75; 25*2=50
n_input=75
#learning rate for adam
learning_rate=0.001

n_classes=2
#size of batch
batch_size=1

In [7]:
d = np.concatenate(data, axis=0).reshape((-1,time_steps*75))
l = np.concatenate(labels, axis=0).reshape((-1,2))

In [8]:
print(d.shape,l.shape)
print(d)
print(l)

(45, 5625) (45, 2)
[[6.20267e+02 1.63590e+02 8.95306e-01 ... 1.14016e+02 4.23289e+02
  7.62022e-01]
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 4.84685e+02 4.02556e+02
  3.66563e-01]
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 1.72753e+02 3.90736e+02
  8.11601e-01]
 ...
 [3.74695e+00 2.38051e+02 3.28673e-01 ... 3.09853e+02 4.11574e+02
  7.55411e-01]
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 4.84759e+02 3.90726e+02
  7.22731e-01]
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 6.70861e+01 4.32371e+02
  8.27393e-01]]
[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]


### TODO:
- remove confidence
- Normalize/Standerdize

In [9]:
testSize=10 
test_data = d[:testSize,:]
test_labels = l[:testSize,:]
train_data = d[testSize:,:]
train_labels = l[testSize:,:]

In [10]:
test_data.shape, test_labels.shape

((10, 5625), (10, 2))

In [11]:
train_data.shape, train_labels.shape

((35, 5625), (35, 2))

In [12]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

In [13]:
#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

In [14]:
#defining the network
lstm_layer=tf.nn.rnn_cell.LSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

In [15]:
#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

In [16]:
#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [17]:
#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<=30:
        batch_x=train_data[batch_size*iter:batch_size*(iter+1)]

        batch_y=train_labels[batch_size*iter:batch_size*(iter+1)]
        #print(batch_x)
        batch_x=batch_x.reshape((batch_size,time_steps,n_input))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %5==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1

    #calculating test accuracy
    test_data = test_data[:batch_size].reshape((-1, time_steps, n_input))
    test_label = test_labels[:batch_size]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

For iter  5
Accuracy  1.0
Loss  0.03886436
__________________
For iter  10
Accuracy  1.0
Loss  1.060957e-05
__________________
For iter  15
Accuracy  1.0
Loss  0.0017353727
__________________
For iter  20
Accuracy  1.0
Loss  0.0012401993
__________________
For iter  25
Accuracy  1.0
Loss  0.05737934
__________________
For iter  30
Accuracy  1.0
Loss  2.3841855e-07
__________________
Testing Accuracy: 1.0


In [18]:
sess.close()

# Visualisation

In [19]:
show_data = d.reshape(-1,n_input)

In [20]:
show_data.shape[0]

3375

In [21]:
testing = show_data[1]
testing[1]

163.59

In [23]:
import pygame

# Initialize Pygame.
pygame.init()
# Set size of pygame window.
screen=pygame.display.set_mode((640,480))
# Create empty pygame surface.
background = pygame.Surface(screen.get_size())
# Fill the background white color.
background.fill((255, 255, 255))
# Convert Surface object to make blitting faster.
background = background.convert()
# Copy background to screen (position (0, 0) is upper left corner).
screen.blit(background, (0,0))
# Create Pygame clock object.
clock = pygame.time.Clock()

mainloop = True
# Desired framerate in frames per second. Try out other values.
FPS = 30
# How many seconds the "game" is played.
playtime = 0.0

count = 0.0

speed = 1

try:
    
    while mainloop:
        # Do not go faster than this framerate.
        milliseconds = clock.tick(FPS*speed)
        playtime += milliseconds / 1000.0

        for event in pygame.event.get():
            # User presses QUIT-button.
            if event.type == pygame.QUIT:
                mainloop = False
            elif event.type == pygame.KEYDOWN:
                # User presses ESCAPE-Key
                if event.key == pygame.K_ESCAPE:
                    mainloop = False

        # Print framerate and playtime in titlebar.
        text = "FPS: {0:.2f}   Playtime: {1:.2f}".format(clock.get_fps(), playtime)
        pygame.display.set_caption(text)
        background.fill((255, 255, 255))
        f = show_data[int(count)%(show_data.shape[0])]
        for i in range(0,f.shape[0],3):

            confColor = 255 - int(255*f[i+2])%255
            #pygame.draw.rect(background, (confColor, confColor, confColor), (int(f[i]),int(f[i+1]),4,4)) #bw
            pygame.draw.rect(background, (confColor, 0, 255-confColor), (int(f[i]),int(f[i+1]),4,4)) #rb
        screen.blit(background, (300-f[8*3],300-f[8*3+1]))
        #Update Pygame display.
        pygame.display.flip()

        count += 1
except TypeError:
    print("Type error: check show_DATA", count," ", i)
except:
    print("unknown error")
else:
    pygame.quit()
# Finish Pygame.
pygame.quit()

# At the very last:
print("This game was played for {0:.2f} seconds".format(playtime))

This game was played for 17.37 seconds


# Live Data-read

In [25]:
import sys
import time
import logging
from watchdog.observers import Observer
from watchdog.events import LoggingEventHandler
from watchdog.events import PatternMatchingEventHandler  

In [26]:
class MyHandler(PatternMatchingEventHandler):
    patterns = ["*.json"]
    data = []

    def process(self, event):
        """
        event.event_type 
            'modified' | 'created' | 'moved' | 'deleted'
        event.is_directory
            True | False
        event.src_path
            path/to/observed/file
        """
        # the file will be processed there
        try:
            #print(i)
            f = pd.read_json(event.src_path).iloc[0,0]['pose_keypoints_2d']
            self.data.append(f)
            print(f)
        except ValueError:
            print("ERROR reading: ",event.src_path, event.event_type)


    def on_created(self, event):
        self.process(event)
        
observer = Observer()
observer.schedule(MyHandler(), '.')
observer.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()